In [ ]:
# Optional to download multiple layers conveniently

from huggingface_hub import snapshot_download
import os

repo_id = "canrager/lm_sae"

release_names = [
    "sae_bench_gemma-2-2b_sweep_standard_ctx128_ef2_0824",
    "sae_bench_gemma-2-2b_sweep_standard_ctx128_ef8_0824",
    "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef2_0824",
    "sae_bench_gemma-2-2b_sweep_topk_ctx128_ef8_0824",
    "sae_bench_pythia70m_sweep_gated_ctx128_0730",
    "sae_bench_pythia70m_sweep_panneal_ctx128_0730",
    "sae_bench_pythia70m_sweep_standard_ctx128_0712",
    "sae_bench_pythia70m_sweep_topk_ctx128_0730",
]

# for release_name in release_names:
#     folder_name = release_name.replace("sae_bench_", "")

#     downloaded_dir = snapshot_download(
#         repo_id, allow_patterns=[f"{folder_name}/*"], ignore_patterns=["*ae.pt"], local_dir="", force_download=True
#     )

#     print(f"Folder downloaded to {downloaded_dir}")

In [ ]:
from typing import Optional
import json


def get_nested_folders(path: str) -> list[str]:
    """
    Recursively get a list of folders that contain an ae.pt file, starting the search from the given path
    """
    folder_names = []

    # We use config.json so it also works for data folders
    for root, dirs, files in os.walk(path):
        if "config.json" in files:
            folder_names.append(root)

    return folder_names


def check_for_empty_folders(ae_group_paths: list[str]) -> bool:
    """So your run doesn't crash / do nothing interesting because folder 13 is empty."""
    for ae_group_path in ae_group_paths:
        if len(get_nested_folders(ae_group_path)) == 0:
            raise ValueError(f"No folders found in {ae_group_path}")
    return True


def get_ae_group_paths(
    dictionaries_path: str, sweep_name: str, submodule_trainers: Optional[dict]
) -> list[str]:
    if submodule_trainers is None:
        return [f"{dictionaries_path}/{sweep_name}"]

    ae_group_paths = []

    for submodule in submodule_trainers.keys():
        trainer_ids = submodule_trainers[submodule]["trainer_ids"]

        base_filename = f"{dictionaries_path}/{sweep_name}/{submodule}"

        if trainer_ids is None:
            ae_group_paths.append(base_filename)
        else:
            for trainer_id in trainer_ids:
                ae_group_paths.append(f"{base_filename}/trainer_{trainer_id}")

    check_for_empty_folders(ae_group_paths)

    return ae_group_paths


def get_ae_paths(ae_group_paths: list[str]) -> list[str]:
    ae_paths = []
    for ae_group_path in ae_group_paths:
        ae_paths.extend(get_nested_folders(ae_group_path))
    return ae_paths


for release_name in release_names:
    folder_name = release_name.replace("sae_bench_", "")
    ae_group_paths = get_ae_group_paths(".", folder_name, None)
    ae_paths = get_ae_paths(ae_group_paths)

    release_results_dict = {"sae_config_dictionary_learning": {}, "basic_eval_results": {}}

    for ae_path in ae_paths:
        print(ae_path)

        config_path = os.path.join(ae_path, "config.json")
        eval_results_path = os.path.join(ae_path, "eval_results.json")

        with open(config_path, "r") as f:
            config = json.load(f)

        with open(eval_results_path, "r") as f:
            eval_results = json.load(f)

        ae_name = ae_path.replace("./", "")

        release_results_dict["sae_config_dictionary_learning"][ae_name] = config
        release_results_dict["basic_eval_results"][ae_name] = eval_results

    with open(f"{release_name}_data.json", "w") as f:
        json.dump(release_results_dict, f, indent=4)

In [ ]:
# Optional to download multiple layers conveniently

from huggingface_hub import snapshot_download
import os

repo_id = "adamkarvonen/gemma-2-2b-jumprelu"


release_names = ["gemma-2-2b_sweep_jumprelu_0902"]

for release_name in release_names:
    folder_name = release_name.replace("sae_bench_", "")

    downloaded_dir = snapshot_download(
        repo_id,
        allow_patterns=[f"*"],
        ignore_patterns=["*ae.pt"],
        local_dir="",
        force_download=True,
    )

    print(f"Folder downloaded to {downloaded_dir}")

In [ ]:
from typing import Optional
import json
import os
import re


release_names = ["gemma-2-2b_sweep_jumprelu_0902"]

l0_dict = {
    3: [14, 28, 59, 142, 315],
    7: [20, 36, 69, 137, 285],
    11: [22, 41, 80, 168, 393],
    15: [23, 41, 78, 150, 308],
    19: [23, 40, 73, 137, 279]
}


def extract_trainer_and_layer(input_string: str) -> tuple[int, int]:
    pattern = r"gemma-\d+-\d+b_sweep_jumprelu_\d+/resid_post_layer_(\d+)/trainer_(\d+)"
    match = re.search(pattern, input_string)

    if match:
        layer_num = int(match.group(1))
        trainer_num = int(match.group(2))
        return trainer_num, layer_num
    else:
        raise ValueError("Input string does not match the expected format")


def get_nested_folders(path: str) -> list[str]:
    """
    Recursively get a list of folders that contain an ae.pt file, starting the search from the given path
    """
    folder_names = []

    # We use config.json so it also works for data folders
    for root, dirs, files in os.walk(path):
        if "config.json" in files:
            folder_names.append(root)

    return folder_names


def check_for_empty_folders(ae_group_paths: list[str]) -> bool:
    """So your run doesn't crash / do nothing interesting because folder 13 is empty."""
    for ae_group_path in ae_group_paths:
        if len(get_nested_folders(ae_group_path)) == 0:
            raise ValueError(f"No folders found in {ae_group_path}")
    return True


def get_ae_group_paths(
    dictionaries_path: str, sweep_name: str, submodule_trainers: Optional[dict]
) -> list[str]:
    if submodule_trainers is None:
        return [f"{dictionaries_path}/{sweep_name}"]

    ae_group_paths = []

    for submodule in submodule_trainers.keys():
        trainer_ids = submodule_trainers[submodule]["trainer_ids"]

        base_filename = f"{dictionaries_path}/{sweep_name}/{submodule}"

        if trainer_ids is None:
            ae_group_paths.append(base_filename)
        else:
            for trainer_id in trainer_ids:
                ae_group_paths.append(f"{base_filename}/trainer_{trainer_id}")

    check_for_empty_folders(ae_group_paths)

    return ae_group_paths


def get_ae_paths(ae_group_paths: list[str]) -> list[str]:
    ae_paths = []
    for ae_group_path in ae_group_paths:
        ae_paths.extend(get_nested_folders(ae_group_path))
    return ae_paths


for release_name in release_names:
    folder_name = release_name.replace("sae_bench_", "")
    ae_group_paths = get_ae_group_paths(".", folder_name, None)
    ae_paths = get_ae_paths(ae_group_paths)

    print(ae_paths)

    release_results_dict = {"sae_config_dictionary_learning": {}, "basic_eval_results": {}}

    for ae_path in ae_paths:
        print(ae_path)

        config_path = os.path.join(ae_path, "config.json")
        eval_results_path = os.path.join(ae_path, "eval_results.json")

        with open(config_path, "r") as f:
            config = json.load(f)

        with open(eval_results_path, "r") as f:
            eval_results = json.load(f)

        ae_name = ae_path.replace("./", "")

        trainer_id, layer_num = extract_trainer_and_layer(ae_name)

        print(ae_name, trainer_id, layer_num)

        formatted_name = f"layer_{layer_num}/width_16k/average_l0_{l0_dict[layer_num][trainer_id]}"

        print(formatted_name)

        release_results_dict["sae_config_dictionary_learning"][formatted_name] = config
        release_results_dict["basic_eval_results"][formatted_name] = eval_results

    with open(f"gemma-scope-2b-pt-res_data.json", "w") as f:
        json.dump(release_results_dict, f, indent=4)